In [134]:
import os
import pandas as pd
import numpy as np
import pickle
import collections
from Graham.Aggregation import Aggregate
import pandas_datareader.data as web
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.width', 1500)
pd.set_option('display.precision', 2)
pd.options.display.float_format = '{:20,.2f}'.format
import numpy as np

In [52]:
symbols = np.array(os.listdir('/Users/duncangh/PycharmProjects/FSA/data/financials_data/')[1:])

In [137]:
class Standardize:
    def __init__(self, sym, rf=1.5):
        self.symbol = sym
        self.statements = Aggregate(sym).statements
        self.risk_free = rf

        self.balance = self.statements[0] / 10 ** 6
        self.income = self.statements[1] / 10 ** 6
        self.cash = self.statements[2] / 10 ** 6
        
        
        self.current_assets = np.array(['IncomeTaxesReceivable', 'OtherAssetsCurrent',
       'DeferredTaxAssetsLiabilitiesNetCurrent', 'RegulatoryAssetsCurrent',
       'InventoryNet', 'OtherPrepaidExpenseCurrent',
       'InventoryFinishedGoods', 'ReceivablesNetCurrent', 'PrepaidTaxes',
       'InventoryWorkInProcess', 'PrepaidExpenseCurrent',
       'DerivativeAssetsCurrent',
       'AssetsOfDisposalGroupIncludingDiscontinuedOperationCurrent',
       'InventoryRawMaterialsAndSupplies',
       'CashAndCashEquivalentsAtCarryingValue', 'ShortTermInvestments',
       'CapitalLeasedAssetsGross',
       'RestrictedCashAndCashEquivalentsAtCarryingValue',
       'RestrictedCashAndInvestmentsCurrent',
       'CashEquivalentsAtCarryingValue', 'MarketableSecuritiesCurrent',
       'MarketableSecuritiesCurrent', 'NotesAndLoansReceivableNetCurrent',
       'AccountsReceivableNetCurrent',
       'AvailableForSaleSecuritiesDebtSecuritiesCurrent',
       'AvailableForSaleSecuritiesDebtSecuritiesCurrent',
       'AvailableForSaleSecuritiesCurrent',
       'PrepaidExpenseAndOtherAssetsCurrent'])

        self.current_liabilities = np.array(['CostsInExcessOfBillingsOnUncompletedContractsOrProgramsExpectedToBeCollectedWithinOneYear',
       'LiabilitiesOfDisposalGroupIncludingDiscontinuedOperationCurrent',
       'OtherLiabilitiesCurrent', 'DueToRelatedPartiesCurrent',
       'RegulatoryLiabilityCurrent',
       'DeferredTaxLiabilitiesCurrent', 'AccountsPayableCurrent',
       'DebtCurrent', 'AccountsPayableAndAccruedLiabilitiesCurrent',
       'AccruedLiabilitiesCurrent', 'EmployeeRelatedLiabilitiesCurrent','CustomerAdvancesCurrent',
       'DeferredRevenueCurrent', 'DeferredRevenueAndCreditsCurrent',
       'DerivativeLiabilitiesCurrent', 'OtherShortTermBorrowings',
       'BankOverdrafts', 'CommercialPaper', 'TaxesPayableCurrent',
       'ShortTermBankLoansAndNotesPayable',
       'CapitalLeaseObligationsCurrent', 'ShortTermBorrowings',
       'LongTermDebtCurrent'])
        
    
        self.interest_bearing = np.array(['DueToRelatedPartiesCurrent', 'DebtCurrent',
       'OtherShortTermBorrowings', 'BankOverdrafts', 'CommercialPaper',
       'ShortTermBankLoansAndNotesPayable',
       'CapitalLeaseObligationsCurrent', 'ShortTermBorrowings',
       'LongTermDebtCurrent', 'LongTermDebtAndCapitalLeaseObligations',
       'CapitalLeaseObligationsNoncurrent', 'OtherLongTermDebtNoncurrent',
       'LongTermDebtAndCapitalLeaseObligationsCurrent',
       'LongTermDebtNoncurrent', 'SeniorLongTermNotes',
       'ConvertibleLongTermNotesPayable'], 
          dtype='<U45')
        
        self.fcfe = self.free_cash_flow_equity()

    
    def change_nwc(self):
        if 'AssetsCurrent' and 'LiabilitiesCurrent' in self.balance.index:
            net = self.balance.loc['AssetsCurrent'] - self.balance.loc['LiabilitiesCurrent']
    
        else:
            net = (self.balance[self.balance.index.isin(self.current_assets)].sum()).fillna(0) - (self.balance[self.balance.index.isin(self.current_liabilities)].sum()).fillna(0)
    
        self.chg_wc = net - net.shift(1)
        return self.chg_wc.fillna(0)
    

    def CapEx(self):
        self.capex = self.cash.loc[self.cash.index[self.cash.index.str.contains('PropertyPlantAndE')]]

        if self.capex.empty:
            return 0
        else:
            self.capex = self.capex.loc[self.capex.index.values[0]]
            return self.capex.fillna(0)

    def debt_cash(self):
        issued = self.cash.loc[
            self.cash.index[(self.cash.index.str.contains('Debt')) & (self.cash.index.str.contains('Iss'))]]
        if not issued.empty:
            issued = issued.loc[issued.index[0]].fillna(0)

        payments = self.cash.loc[
            self.cash.index[(self.cash.index.str.contains('Debt')) & (self.cash.index.str.contains('Repay'))]]
        if not payments.empty:
            payments = payments.loc[payments.index[0]].fillna(0)

        if payments.empty:
            return issued

        elif issued.empty:
            return payments

        else:
            return (issued - payments).fillna(0)

    def DeprAmort(self):
        DA = self.cash.loc[
            self.cash.index[(self.cash.index.str.contains('Depre')) | (self.cash.index.str.contains('Amort'))]]
        if not DA.empty:
            return DA.max()
        else:
            return 0

    def interest_expense(self):
        # Handle all known cases of interest expense
        int_exp = self.income.loc[self.income.index[(self.income.index.str.contains('Interest'))
                                                    & (~self.income.index.str.contains('Minor'))
                                                    & (~self.income.index.str.contains('controll'))
                                                    & (~self.income.index.str.contains('Before'))
                                                    & (~self.income.index.str.contains('Gain'))
                                                    & (~self.income.index.str.contains('TotalRevenues'))
                                                    & (~self.income.index.str.contains('Investment'))
                                                    & (~self.income.index.str.contains('Nonopera'))
                                                    & (~self.income.index.str.contains('Income'))].values[0]]

        if not int_exp.empty:
            return int_exp.fillna(0)
        else:
            return 0

    def ebit(self):
        """
        Calculate the EBIT

        """
        try:
            EBIT = self.income.loc['OperatingIncomeLoss'].fillna(0)
        except:
            EBIT = self.income.loc[self.income.index[self.income.index.str.contains('OperatingIncome')]].fillna(0)
            if EBIT.empty:
                try:
                    EBIT = self.income.loc['NetIncomeLoss'] + self.income.loc[
                        'IncomeTaxExpenseBenefit'] + self.interest_expense()
                    EBIT = EBIT.fillna(0)
                except:
                    pass

        return EBIT

    def free_cash_flow_equity(self):
        """
        FCFe = NI + D/A - ∆ nwc - CapEx  + Net Debt
        """

        try:
            NI = self.income.loc['NetIncomeLoss'].fillna(0)
        except:
            try:
                NI = self.cash.loc['NetIncomeLoss'].fillna(0)
            except:
                NI = self.income.loc[self.income.index[self.income.index.str.contains('NetIncome')].values[0]].fillna(0)

        DA = self.DeprAmort()
        nwc_chg = self.change_nwc()
        capex = self.CapEx()

        net_debt = self.debt_cash()
        if net_debt.empty:
            net_debt = 0

        NIDA = ((NI + DA).fillna(0) + net_debt).fillna(0)
        nwcap = (nwc_chg + capex).fillna(0)
        return (NIDA - nwcap)

    def free_cash_flow_firm(self):
        """
        FCFf = EBIT + Interest Expense + D/A - ∆ nwc - CapEx
        """

        EBIT = self.ebit()

        DA = self.DeprAmort()
        nwc_chg = self.change_nwc()
        capex = self.CapEx()

        int_exp = self.interest_expense()

        EBITDA = ((EBIT + DA).fillna(0) + int_exp).fillna(0)
        nwcap = (nwc_chg + capex).fillna(0)
        return (EBITDA - nwcap)
    
    def degree_of_operating_leverage(self):
        """
              Percent Change in EBIT
        DOL =  --------------------
              Percent Change in Sales
          
        """
        return self.income.loc['Revenues'].pct_change() / self.ebit().pct_change()

    
    def book_value_debt(self):
        return self.balance[self.balance.index.isin(self.interest_bearing)].sum()
    

    def book_value_equity(self):
        """
        Book Value of Equity = Value of Stockholders Equity

        """
        return self.balance.loc['StockholdersEquity'].fillna(0)

    def cost_debt(self, debt):
        """
        Calculate the cost of debt Using
            1. amount of debt
            2. interest expense
            3. Tax Rate if possible

        """
        interest = self.interest_expense()
        earnings_before_taxes = (self.ebit() - interest).fillna(0)
        tax_rate = self.income.loc['IncomeTaxExpenseBenefit'].fillna(0) / earnings_before_taxes

        interest_rate = (interest / debt) * 100

        if 0 < tax_rate.mean() < 1:
            return (interest_rate * (1 - tax_rate)).fillna(0)
        else:
            return interest_rate.fillna(0)

    def cost_equity(self):
        """
        Rrf + ß(Rm - Rrf)

        MRP is assumed to be 4.5%
        """
#         beta = pd.read_pickle('/Users/duncangh/PycharmProjects/FSA/Graham/Beta.Pkl')['SPY'].abs()
#         beta.index = beta.index.astype(str)
        beta = self.beta()
        return self.risk_free + beta * (4.5 - self.risk_free)
    
    def beta(self):
        try:
            end = pd.to_datetime('today')
            start = end - pd.to_timedelta('1080D') # 3 years
            stocks = web.get_data_yahoo(['^GSPC', self.symbol], start, end)['Adj Close']

            beta = (stocks.pct_change().cov() / stocks.pct_change().var()).loc[self.symbol, '^GSPC']
        except:
            beta = 1
        return beta

    def weighted_average_cost_of_capital(self):
        """
        Total Capital = BV(equity) + BV(debt)
        Book Value of Equity = Assets - Liabilities
        Book Value of Debt = (Assets - Equity) - Non-Interest Bearing Obligations

        Equity Cost = Rrf + ß(Rm - Rrf)
        Cost of Debt = (Interest Expense / Book Value of Debt) * (1 - MarginalTaxRate)

        Weight of Equity = BV Equity / Total Capital
        Weight of Debt = BV of Debt / Total Capital

        WACC = (WeightofEquity * CostofEquity) + (WeightofDebt * CostofDebt)
        WACC = rD(1 - Tc) * (D / V) + rE * (E / V)

        :return: WACC
        """
        book_value_debt = self.book_value_debt()
        book_value_equity = self.book_value_equity()

        total_capital = book_value_debt + book_value_equity

        weight_debt = book_value_debt / total_capital
        weight_equity = book_value_equity / total_capital

        cost_debt = self.cost_debt(book_value_debt)
        cost_equity = self.cost_equity()

        wacc = (weight_debt * cost_debt) + (weight_equity * cost_equity)

        return weight_debt, weight_equity, cost_debt, cost_equity, wacc

    def make_model(self):
        wd, we, cd, ce, wacc = self.weighted_average_cost_of_capital()
        fcfe = self.free_cash_flow_equity()
        fcff = self.free_cash_flow_firm()
        model = pd.DataFrame({'FCFF': fcff, 'FCFE': fcfe})
        model = model.assign(Cost_Equity=ce, Cost_Debt=cd, Weight_Equity=we, Weight_Debt=wd, WACC=wacc)
        model = model.fillna(method='bfill')
        return model

In [140]:
success = []
for sym in symbols:
    try:
        Standardize(sym).make_model()
        success.append(sym)
    except:
        pass
success

['ADS',
 'AFL',
 'AGN',
 'AMZN',
 'AON',
 'BBY',
 'BEN',
 'BHI',
 'BLL',
 'CL',
 'CRM',
 'CXO',
 'D',
 'DLPH',
 'EBAY',
 'ED',
 'ETFC',
 'EXPD',
 'FLS',
 'GPC',
 'GWW',
 'HCP',
 'HST',
 'HUM',
 'INTU',
 'JEC',
 'KHC',
 'LH',
 'MMM',
 'NFLX',
 'NVDA',
 'ORCL',
 'TSLA']

In [141]:
np.array(success)

array(['ADS', 'AFL', 'AGN', 'AMZN', 'AON', 'BBY', 'BEN', 'BHI', 'BLL',
       'CL', 'CRM', 'CXO', 'D', 'DLPH', 'EBAY', 'ED', 'ETFC', 'EXPD',
       'FLS', 'GPC', 'GWW', 'HCP', 'HST', 'HUM', 'INTU', 'JEC', 'KHC',
       'LH', 'MMM', 'NFLX', 'NVDA', 'ORCL', 'TSLA'], 
      dtype='<U4')

In [139]:
s = Standardize('AMZN')
s.make_model()

,FCFE,FCFF,Cost_Debt,Cost_Equity,WACC,Weight_Debt,Weight_Equity
2007,837.00,978.00,0.00,5.09,4.51,0.27,0.73
2008,"1,030.00","1,200.00",0.00,5.09,4.51,0.27,0.73
2009,"1,367.00","1,541.00",0.00,5.09,4.51,0.27,0.73
2010,"2,200.00","2,350.00",0.00,5.09,4.51,0.27,0.73
2011,"1,891.00","2,010.00",0.00,5.09,4.51,0.27,0.73
2012,"1,713.00",-858.00,2.98,5.09,4.51,0.27,0.73
2013,"1,126.00","1,344.00",4.42,5.09,4.92,0.25,0.75
2014,"4,378.00","-1,352.00",2.54,5.09,3.98,0.43,0.57
2015,"3,304.00","5,047.00",5.57,5.09,5.27,0.38,0.62


In [120]:

def degree_of_operating_leverage(standard):
    """
          Percent Change in EBIT
    DOL =  --------------------
          Percent Change in Sales
          
    """
    return standard.income.loc['Revenues'].pct_change() / standard.ebit().pct_change()
degree_of_operating_leverage(tsla) * 100

2009                    nan
2010                   2.34
2011                 105.16
2012                 180.23
2013                -458.49
2014                  28.76
2015                   9.34
dtype: float64

In [121]:
tsla = Standardize('TSLA')
tsla.income.loc[['Revenues', 'OperatingExpenses', 'OtherNonoperatingIncomeExpense']]

,2009,2010,2011,2012,2013,2014,2015
Fiel,,,,,,,
Revenues,111.94,116.74,204.24,413.26,"2,013.50","3,198.36","4,046.03"
OperatingExpenses,61.43,177.57,313.08,424.35,517.54,"1,068.36","1,640.13"
OtherNonoperatingIncomeExpense,-1.45,-6.58,-2.65,-1.83,22.60,1.81,-41.65


In [124]:
tsla.income.loc[['Revenues', 'OperatingExpenses', 'OperatingIncomeLoss']].pct_change(axis=1)

,2009,2010,2011,2012,2013,2014,2015
Fiel,,,,,,,
Revenues,nan,0.04,0.75,1.02,3.87,0.59,0.27
OperatingExpenses,nan,1.89,0.76,0.36,0.22,1.06,0.54
OperatingIncomeLoss,nan,1.83,0.71,0.57,-0.84,2.05,2.84


In [109]:
tsla = Standardize('TSLA')
income = tsla.income

In [110]:
tsla.ebit()

2009                 -51.90
2010                -146.84
2011                -251.49
2012                -394.28
2013                 -61.28
2014                -186.69
2015                -716.63
Name: OperatingIncomeLoss, dtype: float64

In [ ]:
debt_cash = ['RepaymentOfNotesReceivableFromRelatedParties',
       'InterestPaidCapitalized', 'GainsLossesOnExtinguishmentOfDebt',
       'ProceedsFromRepaymentsOfLongTermDebtAndCapitalSecurities',
       'ProceedsFromRepaymentsOfCommercialPaper',
       'ProceedsFromRepaymentsOfBankOverdrafts', 'ProceedsFromOtherDebt',
       'PaymentsForProceedsFromLoansReceivable', 'PaymentsOfLoanCosts',
       'RepaymentsOfOtherDebt', 'PaymentsOfDebtIssuanceCosts',
       'ProceedsFromRepaymentsOfShortTermDebt',
       'ProceedsFromShortTermDebt', 'RepaymentsOfLinesOfCredit',
       'ProceedsFromLinesOfCredit',
       'RepaymentsOfLongTermDebtAndCapitalSecurities',
       'RepaymentsOfShortTermDebt',
       'ProceedsFromShortTermDebtMaturingInMoreThanThreeMonths',
       'ProceedsFromRepaymentsOfShortTermDebtMaturingInThreeMonthsOrLess',
       'RepaymentsOfLongTermCapitalLeaseObligations',
       'ProceedsFromIssuanceOfSecuredDebt',
       'RepaymentsOfLongTermLinesOfCredit',
       'ProceedsFromIssuanceOfSeniorLongTermDebt',
       'RepaymentsOfUnsecuredDebt', 'ProceedsFromIssuanceOfLongTermDebt',
       'ProceedsFromLongTermLinesOfCredit',
       'ProceedsFromMaturitiesPrepaymentsAndCallsOfAvailableForSaleSecurities',
       'RepaymentsOfLongTermDebt', 'RepaymentsOfSecuredDebt',
       'RepaymentsOfAssumedDebt', 'RepaymentsOfOtherLongTermDebt',
       'RepaymentsOfConvertibleDebt',
       'ProceedsFromIssuanceOfUnsecuredDebt',
       'ProceedsFromConvertibleDebt',
       'ProceedsFromSaleOfAvailableForSaleSecuritiesDebt',
       'RepaymentsOfSeniorDebt']